# 特征工程技术

搬运参考：https://www.kaggle.com/c/ieee-fraud-detection/discussion/108575

## 关于编码
在执行编码时，最好训练和测试集一起编码，如下所示

In [ ]:
df = pd.concat([train[col],test[col]],axis=0)
# PERFORM FEATURE ENGINEERING HERE
train[col] = df[:len(train)]
test[col] = df[len(train):]

## NAN值加工
如果将np.nan给LGBM，那么在每个树节点分裂时，它会分裂非 NAN 值，然后将所有 NAN 发送到左节点或右节点，这取决于什么是最好的。

因此，NAN 在每个节点都得到特殊处理，并且可能会变得过拟合。

通过简单地将所有 NAN 转换为低于所有非 NAN 值的负数（例如 - 999），来防止测试集过拟合。

In [ ]:
df[col].fillna(-999, inplace=True)

这样LGBM将不再过度处理 NAN。相反，它会给予它与其他数字相同的关注。可以尝试两种方法，看看哪个给出了最高的CV。

## 标签编码/因式分解/内存减少
标签编码（分解）将（字符串、类别、对象）列转换为整数。类似get_dummies，不同点在于如果有几十个取值，如果用pd.get_dummies()则会得到好几十列，增加了数据的稀疏性

In [1]:
import numpy as np
import pandas as pd
df = pd.DataFrame(['green','bule','red','bule','green'],columns=['color'])
df['color'],_ = df['color'].factorize()
df

,color
0,0
1,1
2,2
3,1
4,0


之后，可以将其转换为 int8、int16 或 int32用以减少内存，具体取决于 max 是否小于 128、小于 32768。

In [2]:
if df['color'].max()<128:
    df['color'] = df['color'].astype('int8')
elif df['color'].max()<32768:
    df['color'] = df['color'].astype('int16')
else: df['color'] = df['color'].astype('int32')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype
---  ------  --------------  -----
 0   color   5 non-null      int8 
dtypes: int8(1)
memory usage: 133.0 bytes


In [3]:
df['color'] = df['color'].astype('int32')  # 如果使用int32，可以看到memory usage: 变成148了
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype
---  ------  --------------  -----
 0   color   5 non-null      int32
dtypes: int32(1)
memory usage: 148.0 bytes


另外为了减少内存，人们memory_reduce在其他列上使用流行的功能。

一种更简单、更安全的方法是将所有 float64 转换为 float32，将所有 int64 转换为 int32。（最好避免使用 float16。如果你愿意，可以使用 int8 和 int16）。

In [4]:
for col in df.columns:
    if df[col].dtype=='float64': df[col] = df[col].astype('float32')
    if df[col].dtype=='int64': df[col] = df[col].astype('int32')

## 分类特征
对于分类变量，可以选择告诉 LGBM 它们是分类的（但内存会增加），或者可以告诉 LGBM 将其视为数字（首先需要对其进行标签编码）

In [10]:
df = pd.DataFrame(['green','bule','red','bule','green'],columns=['color'])
df['color'],_ = df['color'].factorize()
df['color'] = df['color'].astype('category')  # 转成分类特征并查看内存使用情况（已知int8内存使用是: 133.0 bytes）
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype   
---  ------  --------------  -----   
 0   color   5 non-null      category
dtypes: category(1)
memory usage: 265.0 bytes


## Splitting
可以通过拆分将单个（字符串或数字）列分成两列。

例如，id_30诸如"Mac OS X 10_9_5"之类的字符串列可以拆分为操作系统"Mac OS X"和版本"10_9_5"。或者例如数字"1230.45"可以拆分为元" 1230"和分"45"。LGBM 无法单独看到这些片段，需要将它们拆分。

## 组合/转化/交互
两个（字符串或数字）列可以合并为一列。例如card1，card2可以成为一个新列

In [ ]:
df['uid'] = df[‘card1’].astype(str)+’_’+df[‘card2’].astype(str)

这有助于LGBM将card1和card2一起去与目标关联，并不会在树节点分裂他们。

但这种uid = card1_card2可能与目标相关，现在LGBM会将其拆分。数字列可以与加法、减法、乘法等组合使用。一个数字示例是

In [ ]:
df['x1_x2'] = df['x1'] * df['x2']